What the data looks like 

for each episode there is a json-file with following structure

{"results":[{"alternatives":[{"transcript":"Hi, this is transcript of first segment","confidence":0.91,"words":[{
                     "startTime":"0s",
                     "endTime":"0.300s",
                     "word":"Hi,"
                  },
                  {"startTime":"0.500s",
                     "endTime":"0.900s",
                     "word":"this"}]}]}]}




What I want the data to look like (kind of)

for each show there shoud be one dict containing show id, show name and all episodes of show split into segments

[{'show-id':'123','show-name':'alex & sigge','episodes':[{'name':'first episode','date':'2020-01-01','segmentTranscripts':[{'segNo':0,'startTime':'00:00s','endTime':'00:20s','transcript':'......',},{},]}]},{},]

- show name would have to be looked up in the metadata.tsv

In [1]:
import json

#for each episode extract {word:'hello',timestamp:'00.00s'} 
    

def extract_segments(path):
    with open(path, "r") as read_file:
        episode = json.load(read_file)
    segments=[]
    #had to do "manual" iteration due to irregularities in data
    iter=0
    for segment in episode["results"]:
        seg_result={}
        #make sure there is only one dict in this list (should be true according to dataset description)
        assert len(segment["alternatives"])==1
        segment_dict=segment["alternatives"][0]
        #sometimes "alternatives" dict is empty...
        if "words" and "transcript"  in segment_dict:
            #add segment number
            seg_result["segNum"]=iter
            #add timestamp of the first word in this segment
            seg_result["startTime"]=segment_dict["words"][0]["startTime"]
            #add timestamp of the last word in this segment
            seg_result["endTime"]=segment_dict["words"][-1]["endTime"]
            #add transcript of this segment 
            seg_result["transcript"]=segment_dict["transcript"]
            segments.append(seg_result)
            iter+=1

    return segments


In [2]:
from training_data_collection import collect_training_episodes

input_file = '../data/podcasts_2020_train.1-8.qrels.txt'
root_dir = '../data/podcasts-no-audio-13GB'
training_episodes = collect_training_episodes(root_dir, input_file)
training_segments = []
for episode in training_episodes:
    training_segments.append(extract_segments(episode))

with open('../data/training_sub.json', 'w') as fout:
        json.dump(training_segments , fout)

344



The Physics of Black Holes	In this episode Zhao fan, Christoff, and Pascale discuss the physics behind the recently released photo of a black hole in the M87 galaxy.
"""
<topic>
<num>3</num>
<query>black hole image</query>
<type>topical</type>
<description>In May 2019 astronomers released the first-ever picture of a black hole.  I would like to hear some conversations and educational discussion about the science of astronomy, black holes, and of the picture itself.</description>
</topic>
"""

In [6]:

from sentence_transformers import SentenceTransformer
import scipy.spatial
import numpy as np


episode_path="/Users/Simpan/spotify/spotify-podcasts-2020/podcasts-transcripts/7/K/show_7KjIumXpDzE8vj47JUpInQ/58608SLCnt70sfgrI3gJZK.json"

embedder = SentenceTransformer('bert-base-nli-mean-tokens')
segments_l=[dict["transcript"] for dict in extract_segments(episode_path)]
#extract_segments(episode_path)

In [7]:
segments_l

["I said get it out now or something to be doing that later. Nobody wants that. Did you hit the Wow Let's Go Kristoff. Hi, welcome to the physics of almost everything podcast where we talk about.",
 'thing physics',
 " Oh, I'm Pascal. I'm Joe Van and I'm Christoph. They were talking about black holes. There's face. You better watch out. Um black holes are actually planes in space in which the gravitational pull is so strong that it can even suck in lights. Yeah, though, they're called black holes. They actually created when a star dies and a supernova is produced which is a large explosion and then when it come packs and comes back together in",
 " Relatively small space relatively small compared to the star that it was before the large amount of density and mass of that volume actually causes it to collapse in on itself and then sucks everything else in there are also three types of black holes miniature which are Fun Size Stellar which are normal and supermassive which you can assume

In [ ]:
import os

base_dir="/Users/Simpan/spotify/spotify-podcasts-2020/podcasts-transcripts"
data=[]
for sub_dir in os.listdir(base_dir):
    for sub_sub_dir in os.listdir(base_dir+'/'+sub_dir):
        for show_dir in os.listdir(base_dir+'/'+sub_dir+'/'+sub_sub_dir):
            show_dict={}
            show_dict["showId"]=show_dir
            show_dict["episodes"]=[]
            for i,episode in enumerate(os.listdir(base_dir+'/'+sub_dir+'/'+sub_sub_dir+'/'+show_dir)):
                episode_dict={}
                #remove .json ending
                episode_dict["episodeId"]=episode[:-5]
                episode_dict["episodeNo"]=i
                episode_path=base_dir+'/'+sub_dir+'/'+sub_sub_dir+'/'+show_dir+'/'+episode
                episode_dict["episodeSegments"]=extract_segments(episode_path)
                show_dict["episodes"].append(episode_dict)
            data.append(show_dict)
with open("test.json", "w+") as write_file:
    json.dump(data, write_file)